In [33]:
import torch
import s3prl.hub
import librosa
import torchaudio
import numpy as np

from importlib import reload
import wav2avatar.streaming.hifigan.hifigan_generator as hifigan
import wav2avatar.streaming.hifigan.hifigan_discriminators as hifigan_discriminators
from wav2avatar.streaming.hifigan.wav_ema_dataset import WavEMADataset
import wav2avatar.streaming.hifigan.collate
reload(hifigan)
reload(wav2avatar.streaming.hifigan.collate)
reload(wav2avatar.streaming.hifigan.wav_ema_dataset)
reload(hifigan_discriminators)

<module 'wav2avatar.streaming.hifigan.hifigan_discriminators' from '/home/prabhune/wav2avatar/wav2avatar/streaming/hifigan/hifigan_discriminators.py'>

In [2]:
device = 0
wlm = getattr(s3prl.hub, "wavlm_large")()

/home/prabhune/miniconda3/envs/w2a/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [3]:
wav = "../../inversion/wav/mlk.wav"
sr = 16000
audio, a_sr = torchaudio.load(wav)
audio = audio.to(0)
if audio.shape[0] > 1:
    audio = audio[0]
if a_sr != sr:
    audio = torchaudio.functional.resample(
        audio, orig_freq=a_sr, new_freq=sr
    )
audio = audio.unsqueeze(0)
print(f"Audio loaded: {audio.shape}")

Audio loaded: torch.Size([1, 280918])


In [4]:
wlm_cnn = wlm.model.feature_extractor.to(device)

In [5]:
wlm_cnn(audio).shape

torch.Size([1, 512, 877])

In [34]:
gan = hifigan.HiFiGANGenerator(in_channels=512, out_channels=12, ar_input=600).to(device)

In [25]:
x = wlm_cnn(audio).to(device)
print(x.shape)

collates = [x[:, :, i:i+50] for i in range(0, x.shape[2] // 50 * 50, 25)]

x_collated = torch.concatenate(collates)
print(x_collated.shape)
#xs = [x[:, :, 0:5]] * 5

torch.Size([1, 512, 877])
torch.Size([34, 512, 50])


In [8]:
ar = torch.zeros((x_collated.shape[0], 12, 50)).to(device)
#ars = [ar[:, 0:5]] * 5

print(ar.shape)
#print(len(ars))

torch.Size([34, 12, 50])


In [16]:
audio_feats.shape

torch.Size([12, 512, 50])

In [35]:
audio_feats = audio_feats.to(device)
ema = ema.to(device)
pred = gan(audio_feats, ema)
print(pred.shape)

inputconv: torch.Size([12, 256, 50])
torch.Size([12, 12, 50])


In [15]:
audio.shape[1] / 16000

17.557375

In [51]:
x.shape

In [86]:
msd_mpd = hifigan_discriminators.HiFiGANMultiScaleMultiPeriodDiscriminator().to(device)

/home/prabhune/miniconda3/envs/w2a/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [87]:
len(msd_mpd(pred)[0][0])

14

In [89]:
def prepare_audio(audio_list, start_idxs):
    """Pad a batch of audio"""
    # Prepare audio for training or validation
    # Crop audio
    return torch.concatenate([
        x[:, :, start_idx:start_idx + 256]
        for x, start_idx in zip(audio_list, start_idxs)])

In [91]:
x_collate = prepare_audio([x, x], [0, 511])

In [69]:
x.shape

torch.Size([1, 512, 877])

In [9]:
wav_ema_dataset = WavEMADataset()

Loading files from directories...


In [10]:
wav_ema_dataset.ema_files[:5]

[PosixPath('/data/prabhune/VCTK/p225/mngu0_wlm_est/p225_001_mic1.npy'),
 PosixPath('/data/prabhune/VCTK/p225/mngu0_wlm_est/p225_001_mic2.npy'),
 PosixPath('/data/prabhune/VCTK/p225/mngu0_wlm_est/p225_002_mic1.npy'),
 PosixPath('/data/prabhune/VCTK/p225/mngu0_wlm_est/p225_002_mic2.npy'),
 PosixPath('/data/prabhune/VCTK/p225/mngu0_wlm_est/p225_003_mic1.npy')]

In [11]:
wav_ema_dataset.wav_files[:5]

[PosixPath('/data/all_data/VCTK/p225/wav_16/p225_001_mic1.wav'),
 PosixPath('/data/all_data/VCTK/p225/wav_16/p225_001_mic2.wav'),
 PosixPath('/data/all_data/VCTK/p225/wav_16/p225_002_mic1.wav'),
 PosixPath('/data/all_data/VCTK/p225/wav_16/p225_002_mic2.wav'),
 PosixPath('/data/all_data/VCTK/p225/wav_16/p225_003_mic1.wav')]

In [10]:
data_loader = torch.utils.data.DataLoader(dataset=wav_ema_dataset, batch_size=5, shuffle=True, collate_fn=wav2avatar.streaming.hifigan.collate.car_collate)

audio_feats, ema, ema_ar = next(iter(data_loader))

In [92]:
print(audio_feats.shape)
print(ema_ar.shape)

torch.Size([16, 512, 50])
torch.Size([16, 12, 50])


In [263]:
print(torch.concatenate(audio_feats, dim=2).shape)
print(torch.concatenate(ema, dim=2).shape)

torch.Size([1, 512, 817])
torch.Size([1, 12, 817])


In [210]:
np.load("/data/prabhune/VCTK/p225/mngu0_wlm_est/p225_030_mic1.npy").shape

(208, 12)

In [211]:
audio, sr = torchaudio.load("/data/all_data/VCTK/p225/wav_16/p225_030_mic1.wav")
audio.shape

torch.Size([1, 66817])

In [212]:
66817/16000

4.1760625

In [215]:
208/50

4.16